# Building an end-to-end Speech Recognition model in PyTorch - [AssemblyAI](https://www.assemblyai.com/)

## installing the requirements

In [1]:
# !pip install torchaudio==0.4.0 torch==1.4.0 comet-ml==3.0.2

## Setting up your data pipeline

In [1]:
import os
from comet_ml import Experiment
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
import torch.nn.functional as F
import torchaudio
import numpy as np


def avg_wer(wer_scores, combined_ref_len):
    return float(sum(wer_scores)) / float(combined_ref_len)


def _levenshtein_distance(ref, hyp):
    """Levenshtein distance is a string metric for measuring the difference
    between two sequences. Informally, the levenshtein disctance is defined as
    the minimum number of single-character edits (substitutions, insertions or
    deletions) required to change one word into the other. We can naturally
    extend the edits to word level when calculate levenshtein disctance for
    two sentences.
    """
    m = len(ref)
    n = len(hyp)

    # special case
    if ref == hyp:
        return 0
    if m == 0:
        return n
    if n == 0:
        return m

    if m < n:
        ref, hyp = hyp, ref
        m, n = n, m

    # use O(min(m, n)) space
    distance = np.zeros((2, n + 1), dtype=np.int32)

    # initialize distance matrix
    for j in range(0,n + 1):
        distance[0][j] = j

    # calculate levenshtein distance
    for i in range(1, m + 1):
        prev_row_idx = (i - 1) % 2
        cur_row_idx = i % 2
        distance[cur_row_idx][0] = i
        for j in range(1, n + 1):
            if ref[i - 1] == hyp[j - 1]:
                distance[cur_row_idx][j] = distance[prev_row_idx][j - 1]
            else:
                s_num = distance[prev_row_idx][j - 1] + 1
                i_num = distance[cur_row_idx][j - 1] + 1
                d_num = distance[prev_row_idx][j] + 1
                distance[cur_row_idx][j] = min(s_num, i_num, d_num)

    return distance[m % 2][n]


def word_errors(reference, hypothesis, ignore_case=False, delimiter=' '):
    """Compute the levenshtein distance between reference sequence and
    hypothesis sequence in word-level.
    :param reference: The reference sentence.
    :type reference: basestring
    :param hypothesis: The hypothesis sentence.
    :type hypothesis: basestring
    :param ignore_case: Whether case-sensitive or not.
    :type ignore_case: bool
    :param delimiter: Delimiter of input sentences.
    :type delimiter: char
    :return: Levenshtein distance and word number of reference sentence.
    :rtype: list
    """
    if ignore_case == True:
        reference = reference.lower()
        hypothesis = hypothesis.lower()

    ref_words = reference.split(delimiter)
    hyp_words = hypothesis.split(delimiter)

    edit_distance = _levenshtein_distance(ref_words, hyp_words)
    return float(edit_distance), len(ref_words)


def char_errors(reference, hypothesis, ignore_case=False, remove_space=False):
    """Compute the levenshtein distance between reference sequence and
    hypothesis sequence in char-level.
    :param reference: The reference sentence.
    :type reference: basestring
    :param hypothesis: The hypothesis sentence.
    :type hypothesis: basestring
    :param ignore_case: Whether case-sensitive or not.
    :type ignore_case: bool
    :param remove_space: Whether remove internal space characters
    :type remove_space: bool
    :return: Levenshtein distance and length of reference sentence.
    :rtype: list
    """
    if ignore_case == True:
        reference = reference.lower()
        hypothesis = hypothesis.lower()

    join_char = ' '
    if remove_space == True:
        join_char = ''

    reference = join_char.join(filter(None, reference.split(' ')))
    hypothesis = join_char.join(filter(None, hypothesis.split(' ')))

    edit_distance = _levenshtein_distance(reference, hypothesis)
    return float(edit_distance), len(reference)


def wer(reference, hypothesis, ignore_case=False, delimiter=' '):
    """Calculate word error rate (WER). WER compares reference text and
    hypothesis text in word-level. WER is defined as:
    .. math::
        WER = (Sw + Dw + Iw) / Nw
    where
    .. code-block:: text
        Sw is the number of words subsituted,
        Dw is the number of words deleted,
        Iw is the number of words inserted,
        Nw is the number of words in the reference
    We can use levenshtein distance to calculate WER. Please draw an attention
    that empty items will be removed when splitting sentences by delimiter.
    :param reference: The reference sentence.
    :type reference: basestring
    :param hypothesis: The hypothesis sentence.
    :type hypothesis: basestring
    :param ignore_case: Whether case-sensitive or not.
    :type ignore_case: bool
    :param delimiter: Delimiter of input sentences.
    :type delimiter: char
    :return: Word error rate.
    :rtype: float
    :raises ValueError: If word number of reference is zero.
    """
    edit_distance, ref_len = word_errors(reference, hypothesis, ignore_case,
                                         delimiter)

    if ref_len == 0:
        raise ValueError("Reference's word number should be greater than 0.")

    wer = float(edit_distance) / ref_len
    return wer


def cer(reference, hypothesis, ignore_case=False, remove_space=False):
    """Calculate charactor error rate (CER). CER compares reference text and
    hypothesis text in char-level. CER is defined as:
    .. math::
        CER = (Sc + Dc + Ic) / Nc
    where
    .. code-block:: text
        Sc is the number of characters substituted,
        Dc is the number of characters deleted,
        Ic is the number of characters inserted
        Nc is the number of characters in the reference
    We can use levenshtein distance to calculate CER. Chinese input should be
    encoded to unicode. Please draw an attention that the leading and tailing
    space characters will be truncated and multiple consecutive space
    characters in a sentence will be replaced by one space character.
    :param reference: The reference sentence.
    :type reference: basestring
    :param hypothesis: The hypothesis sentence.
    :type hypothesis: basestring
    :param ignore_case: Whether case-sensitive or not.
    :type ignore_case: bool
    :param remove_space: Whether remove internal space characters
    :type remove_space: bool
    :return: Character error rate.
    :rtype: float
    :raises ValueError: If the reference length is zero.
    """
    edit_distance, ref_len = char_errors(reference, hypothesis, ignore_case,
                                         remove_space)

    if ref_len == 0:
        raise ValueError("Length of reference should be greater than 0.")

    cer = float(edit_distance) / ref_len
    return cer

class TextTransform:
    """Maps characters to integers and vice versa"""
    def __init__(self):
        char_map_str = """
        ' 0
        <SPACE> 1
        a 2
        b 3
        c 4
        d 5
        e 6
        f 7
        g 8
        h 9
        i 10
        j 11
        k 12
        l 13
        m 14
        n 15
        o 16
        p 17
        q 18
        r 19
        s 20
        t 21
        u 22
        v 23
        w 24
        x 25
        y 26
        z 27
        """
        self.char_map = {}
        self.index_map = {}
        for line in char_map_str.strip().split('\n'):
            ch, index = line.split()
            self.char_map[ch] = int(index)
            self.index_map[int(index)] = ch
        self.index_map[1] = ' '

    def text_to_int(self, text):
        """ Use a character map and convert text to an integer sequence """
        int_sequence = []
        for c in text:
            if c == ' ':
                ch = self.char_map['<SPACE>']
            else:
                ch = self.char_map[c]
            int_sequence.append(ch)
        return int_sequence

    def int_to_text(self, labels):
        """ Use a character map and convert integer labels to an text sequence """
        string = []
        for i in labels:
            string.append(self.index_map[i])
        return ''.join(string).replace('<SPACE>', ' ')

train_audio_transforms = nn.Sequential(
    torchaudio.transforms.MelSpectrogram(sample_rate=16000, n_mels=128),
    torchaudio.transforms.FrequencyMasking(freq_mask_param=30),
    torchaudio.transforms.TimeMasking(time_mask_param=100)
)

valid_audio_transforms = torchaudio.transforms.MelSpectrogram()

text_transform = TextTransform()

def data_processing(data, data_type="train"):
    spectrograms = []
    labels = []
    input_lengths = []
    label_lengths = []
    for (waveform, _, utterance, _, _, _) in data:
        if data_type == 'train':
            spec = train_audio_transforms(waveform).squeeze(0).transpose(0, 1)
        elif data_type == 'valid':
            spec = valid_audio_transforms(waveform).squeeze(0).transpose(0, 1)
        else:
            raise Exception('data_type should be train or valid')
        spectrograms.append(spec)
        label = torch.Tensor(text_transform.text_to_int(utterance.lower()))
        labels.append(label)
        input_lengths.append(spec.shape[0]//2)
        label_lengths.append(len(label))

    spectrograms = nn.utils.rnn.pad_sequence(spectrograms, batch_first=True).unsqueeze(1).transpose(2, 3)
    labels = nn.utils.rnn.pad_sequence(labels, batch_first=True)

    return spectrograms, labels, input_lengths, label_lengths


def GreedyDecoder(output, labels, label_lengths, blank_label=28, collapse_repeated=True):
	arg_maxes = torch.argmax(output, dim=2)
	decodes = []
	targets = []
	for i, args in enumerate(arg_maxes):
		decode = []
		targets.append(text_transform.int_to_text(labels[i][:label_lengths[i]].tolist()))
		for j, index in enumerate(args):
			if index != blank_label:
				if collapse_repeated and j != 0 and index == args[j -1]:
					continue
				decode.append(index.item())
		decodes.append(text_transform.int_to_text(decode))
	return decodes, targets


## The Model
Base of of Deep Speech 2 with some personal improvements

In [2]:
class CNNLayerNorm(nn.Module):
    """Layer normalization built for cnns input"""
    def __init__(self, n_feats):
        super(CNNLayerNorm, self).__init__()
        self.layer_norm = nn.LayerNorm(n_feats)

    def forward(self, x):
        # x (batch, channel, feature, time)
        x = x.transpose(2, 3).contiguous() # (batch, channel, time, feature)
        x = self.layer_norm(x)
        return x.transpose(2, 3).contiguous() # (batch, channel, feature, time) 


class ResidualCNN(nn.Module):
    """Residual CNN inspired by https://arxiv.org/pdf/1603.05027.pdf
        except with layer norm instead of batch norm
    """
    def __init__(self, in_channels, out_channels, kernel, stride, dropout, n_feats):
        super(ResidualCNN, self).__init__()

        self.cnn1 = nn.Conv2d(in_channels, out_channels, kernel, stride, padding=kernel//2)
        self.cnn2 = nn.Conv2d(out_channels, out_channels, kernel, stride, padding=kernel//2)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.layer_norm1 = CNNLayerNorm(n_feats)
        self.layer_norm2 = CNNLayerNorm(n_feats)

    def forward(self, x):
        residual = x  # (batch, channel, feature, time)
        x = self.layer_norm1(x)
        x = F.gelu(x)
        x = self.dropout1(x)
        x = self.cnn1(x)
        x = self.layer_norm2(x)
        x = F.gelu(x)
        x = self.dropout2(x)
        x = self.cnn2(x)
        x += residual
        return x # (batch, channel, feature, time)


class BidirectionalGRU(nn.Module):

    def __init__(self, rnn_dim, hidden_size, dropout, batch_first):
        super(BidirectionalGRU, self).__init__()

        self.BiGRU = nn.GRU(
            input_size=rnn_dim, hidden_size=hidden_size,
            num_layers=1, batch_first=batch_first, bidirectional=True)
        self.layer_norm = nn.LayerNorm(rnn_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.layer_norm(x)
        x = F.gelu(x)
        x, _ = self.BiGRU(x)
        x = self.dropout(x)
        return x


class SpeechRecognitionModel(nn.Module):
    
    def __init__(self, n_cnn_layers, n_rnn_layers, rnn_dim, n_class, n_feats, stride=2, dropout=0.1):
        super(SpeechRecognitionModel, self).__init__()
        n_feats = n_feats//2
        self.cnn = nn.Conv2d(1, 32, 3, stride=stride, padding=3//2)  # cnn for extracting heirachal features

        # n residual cnn layers with filter size of 32
        self.rescnn_layers = nn.Sequential(*[
            ResidualCNN(32, 32, kernel=3, stride=1, dropout=dropout, n_feats=n_feats) 
            for _ in range(n_cnn_layers)
        ])
        self.fully_connected = nn.Linear(n_feats*32, rnn_dim)
        self.birnn_layers = nn.Sequential(*[
            BidirectionalGRU(rnn_dim=rnn_dim if i==0 else rnn_dim*2,
                             hidden_size=rnn_dim, dropout=dropout, batch_first=i==0)
            for i in range(n_rnn_layers)
        ])
        self.classifier = nn.Sequential(
            nn.Linear(rnn_dim*2, rnn_dim),  # birnn returns rnn_dim*2
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(rnn_dim, n_class)
        )

    def forward(self, x):
        x = self.cnn(x)
        x = self.rescnn_layers(x)
        sizes = x.size()
        x = x.view(sizes[0], sizes[1] * sizes[2], sizes[3])  # (batch, feature, time)
        x = x.transpose(1, 2) # (batch, time, feature)
        x = self.fully_connected(x)
        x = self.birnn_layers(x)
        x = self.classifier(x)
        return x


## The Training and Evaluating Script

In [38]:
class IterMeter(object):
    """keeps track of total iterations"""
    def __init__(self):
        self.val = 0

    def step(self):
        self.val += 1

    def get(self):
        return self.val


def train(model, device, train_loader, criterion, optimizer, scheduler, epoch, iter_meter, experiment):
    model.train()
    data_len = len(train_loader.dataset)
    with experiment.train():
        for batch_idx, _data in enumerate(train_loader):

            spectrograms, labels, input_lengths, label_lengths = _data 
            spectrograms, labels = spectrograms.to(device), labels.to(device)

            optimizer.zero_grad()

            output = model(spectrograms)  # (batch, time, n_class)
            output = F.log_softmax(output, dim=2)
            output = output.transpose(0, 1) # (time, batch, n_class)

            loss = criterion(output, labels, input_lengths, label_lengths)
            loss.backward()

            experiment.log_metric('loss', loss.item(), step=iter_meter.get())
            experiment.log_metric('learning_rate', scheduler.get_lr(), step=iter_meter.get())

            optimizer.step()
            scheduler.step()
            iter_meter.step()
            if batch_idx % 100 == 0 or batch_idx == data_len:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(spectrograms), data_len,
                    100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader, criterion, epoch, iter_meter, experiment):
    print('\nevaluating...')
    model.eval()
    test_loss = 0
    test_cer, test_wer = [], []

    with torch.no_grad():
        for i, _data in enumerate(test_loader):

            spectrograms, labels, input_lengths, label_lengths = _data 
            spectrograms, labels = spectrograms.to(device), labels.to(device)

            output = model(spectrograms)  # (batch, time, n_class)
            output = F.log_softmax(output, dim=2)
            output = output.transpose(0, 1) # (time, batch, n_class)

            loss = criterion(output, labels, input_lengths, label_lengths)
            test_loss += loss.item() / len(test_loader)

            decoded_preds, decoded_targets = GreedyDecoder(output.transpose(0, 1), labels, label_lengths)

            print(decoded_preds, decoded_targets)
            for j in range(len(decoded_preds)):
                test_cer.append(cer(decoded_targets[j], decoded_preds[j]))
                test_wer.append(wer(decoded_targets[j], decoded_preds[j]))


    avg_cer = sum(test_cer)/len(test_cer)
    avg_wer = sum(test_wer)/len(test_wer)
    # experiment.log_metric('test_loss', test_loss, step=iter_meter.get())
    # experiment.log_metric('cer', avg_cer, step=iter_meter.get())
    # experiment.log_metric('wer', avg_wer, step=iter_meter.get())

    print('Test set: Average loss: {:.4f}, Average CER: {:4f} Average WER: {:.4f}\n'.format(test_loss, avg_cer, avg_wer))


def main(learning_rate=5e-4, batch_size=20, epochs=10,
        train_url="train-clean-100", test_url="test-clean",
        experiment=Experiment(api_key='dummy_key', disabled=True)):

    hparams = {
        "n_cnn_layers": 3,
        "n_rnn_layers": 5,
        "rnn_dim": 512,
        "n_class": 29,
        "n_feats": 128,
        "stride":2,
        "dropout": 0.1,
        "learning_rate": learning_rate,
        "batch_size": batch_size,
        "epochs": epochs
    }

    experiment.log_parameters(hparams)

    use_cuda = torch.cuda.is_available()
    torch.manual_seed(7)
    device = torch.device("cuda" if use_cuda else "cpu")

    if not os.path.isdir("./data"):
        os.makedirs("./data")

    train_dataset = torchaudio.datasets.LIBRISPEECH("data/librispeech", url=train_url, download=False)
    test_dataset = torchaudio.datasets.LIBRISPEECH("data/librispeech", url=test_url, download=False)

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_loader = data.DataLoader(dataset=train_dataset,
                                batch_size=hparams['batch_size'],
                                shuffle=True,
                                collate_fn=lambda x: data_processing(x, 'train'),
                                **kwargs)
    test_loader = data.DataLoader(dataset=test_dataset,
                                batch_size=hparams['batch_size'],
                                shuffle=False,
                                collate_fn=lambda x: data_processing(x, 'valid'),
                                **kwargs)

    model = SpeechRecognitionModel(
        hparams['n_cnn_layers'], hparams['n_rnn_layers'], hparams['rnn_dim'],
        hparams['n_class'], hparams['n_feats'], hparams['stride'], hparams['dropout']
        ).to(device)

    print(model)
    print('Num Model Parameters', sum([param.nelement() for param in model.parameters()]))

    optimizer = optim.AdamW(model.parameters(), hparams['learning_rate'])
    criterion = nn.CTCLoss(blank=28).to(device)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=hparams['learning_rate'], 
                                            steps_per_epoch=int(len(train_loader)),
                                            epochs=hparams['epochs'],
                                            anneal_strategy='linear')
    
    iter_meter = IterMeter()
    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, criterion, optimizer, scheduler, epoch, iter_meter, experiment)
        test(model, device, test_loader, criterion, epoch, iter_meter, experiment)


In [39]:
learning_rate=5e-4
batch_size=8
epochs=1
train_url="train-clean-100"
test_url="test-clean"
experiment=Experiment(api_key='dummy_key', disabled=True)

hparams = {
    "n_cnn_layers": 3,
    "n_rnn_layers": 5,
    "rnn_dim": 512,
    "n_class": 29,
    "n_feats": 128,
    "stride":2,
    "dropout": 0.1,
    "learning_rate": learning_rate,
    "batch_size": batch_size,
    "epochs": epochs
}

experiment.log_parameters(hparams)

use_cuda = torch.cuda.is_available()
torch.manual_seed(7)
device = torch.device("cuda" if use_cuda else "cpu")

if not os.path.isdir("./data"):
    os.makedirs("./data")

train_dataset = torchaudio.datasets.LIBRISPEECH("data/librispeech", url=train_url, download=False)
test_dataset = torchaudio.datasets.LIBRISPEECH("data/librispeech", url=test_url, download=True)

full_len = len(train_dataset)

train_len = int(0.1 * full_len)
left = full_len - train_len

train_dataset, _ = torch.utils.data.random_split(train_dataset, ( train_len, left))


kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = data.DataLoader(dataset=train_dataset,
                            batch_size=hparams['batch_size'],
                            shuffle=True,
                            collate_fn=lambda x: data_processing(x, 'train'),
                            **kwargs)
test_loader = data.DataLoader(dataset=test_dataset,
                            batch_size=hparams['batch_size'],
                            shuffle=False,
                            collate_fn=lambda x: data_processing(x, 'valid'),
                            **kwargs)

model = SpeechRecognitionModel(
    hparams['n_cnn_layers'], hparams['n_rnn_layers'], hparams['rnn_dim'],
    hparams['n_class'], hparams['n_feats'], hparams['stride'], hparams['dropout']
    ).to(device)


In [40]:
for a in train_loader:
    # spectrograms, labels, input_lengths, label_lengths
    print(a[0].size())
    print(a[1].size())
    print(a[2])
    print(a[3])
    break

torch.Size([8, 1, 128, 1285])
torch.Size([8, 301])
[542, 513, 642, 459, 168, 625, 387, 508]
[195, 148, 301, 227, 67, 247, 105, 193]


In [41]:
train_loader.dataset[0]

(tensor([[-0.0007, -0.0007, -0.0012,  ...,  0.0050,  0.0059, -0.0002]]),
 16000,
 'THUS WE ARE MOVED TO STUDY THE DEVELOPMENT OF SEX LIFE IN THE CHILD ALSO AND WE DISCOVER THE FOLLOWING FROM A NUMBER OF SOURCES IN THE FIRST PLACE IT IS A MISTAKE TO DENY THAT THE CHILD HAS A SEXUAL LIFE AND TO TAKE IT FOR GRANTED',
 8975,
 270782,
 77)

In [42]:
for (waveform, _, utterance, _, _, _) in train_loader.dataset:
    print(waveform, utterance)

    print(train_audio_transforms(waveform).size())
    print(train_audio_transforms(waveform).squeeze(0).transpose(0, 1).size())
    break

tensor([[-0.0007, -0.0007, -0.0012,  ...,  0.0050,  0.0059, -0.0002]]) THUS WE ARE MOVED TO STUDY THE DEVELOPMENT OF SEX LIFE IN THE CHILD ALSO AND WE DISCOVER THE FOLLOWING FROM A NUMBER OF SOURCES IN THE FIRST PLACE IT IS A MISTAKE TO DENY THAT THE CHILD HAS A SEXUAL LIFE AND TO TAKE IT FOR GRANTED
torch.Size([1, 128, 1190])
torch.Size([1190, 128])


In [43]:
def data_processing(data, data_type="train"):
    spectrograms = []
    labels = []
    input_lengths = []
    label_lengths = []
    # (waveform, sample_rate, utterance, speaker_id, chapter_id, utterance_id)
    for (waveform, _, utterance, _, _, _) in data:
        if data_type == 'train':
            spec = train_audio_transforms(waveform).squeeze(0).transpose(0, 1)
        elif data_type == 'valid':
            spec = valid_audio_transforms(waveform).squeeze(0).transpose(0, 1)
        else:
            raise Exception('data_type should be train or valid')
        spectrograms.append(spec)
        label = torch.Tensor(text_transform.text_to_int(utterance.lower()))
        labels.append(label)
        input_lengths.append(spec.shape[0]//2)
        label_lengths.append(len(label))

    spectrograms = nn.utils.rnn.pad_sequence(spectrograms, batch_first=True).unsqueeze(1).transpose(2, 3)
    labels = nn.utils.rnn.pad_sequence(labels, batch_first=True)

    return spectrograms, labels, input_lengths, label_lengths

In [44]:
train_loader.dataset[10]

(tensor([[ 0.0262,  0.0145, -0.0055,  ...,  0.0079, -0.0007, -0.0103]]),
 16000,
 'NOW I DARE SAY YOU ARE THINKING OF MONSIEUR VALANCOURT AND THAT HE MAY HAVE COME AMONG THE ARMIES WHICH THEY SAY ARE COME FROM OUR COUNTRY TO FIGHT AGAINST THIS STATE AND THAT HE HAS MET WITH SOME OF OUR PEOPLE AND IS TAKEN CAPTIVE',
 2007,
 132570,
 15)

In [45]:
# Train
torch.cuda.empty_cache()

print('Num Model Parameters', sum([param.nelement() for param in model.parameters()]))

optimizer = optim.AdamW(model.parameters(), hparams['learning_rate'])
# set blank
criterion = nn.CTCLoss(blank=28).to(device)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=hparams['learning_rate'], 
                                        steps_per_epoch=int(len(train_loader)),
                                        epochs=hparams['epochs'],
                                        anneal_strategy='linear')

iter_meter = IterMeter()
for epoch in range(1, epochs + 1):
    # train(model, device, train_loader, criterion, optimizer, scheduler, epoch, iter_meter, experiment)
    test(model, device, test_loader, criterion, epoch, iter_meter, experiment)

jljlejljljljljejleleljleleljlel', 'ljljbjwljleljejeljljejelejejljljljeljljejljljljeljljlejejljljlejljelejljlejljljljleleleljleleljljlelelelelelel', 'ljljwljejljljljljljljejejljlelejelelelejljljlelelelelelelelelel'] ['the chaos in which his ardour extinguished itself was a cold indifferent knowledge of himself', 'at most by an alms given to a beggar whose blessing he fled from he might hope wearily to win for himself some measure of actual grace', 'well now ennis i declare you have a head and so has my stick', 'on saturday mornings when the sodality met in the chapel to recite the little office his place was a cushioned kneeling desk at the right of the altar from which he led his wing of boys through the responses', 'her eyes seemed to regard him with mild pity her holiness a strange light glowing faintly upon her frail flesh did not humiliate the sinner who approached her', 'if ever he was impelled to cast sin from him and to repent the impulse that moved him was the wish to be her kn

KeyboardInterrupt: 

In [34]:
# torch.save(model.state_dict(), "librispeech/model.pt")

In [12]:
# model.load_state_dict(torch.load("librispeech/model.pt"))

<All keys matched successfully>

In [53]:
torch.cuda.empty_cache()
device = 'cuda'
model.eval()
for _data in train_loader:
    spectrograms, labels, input_lengths, label_lengths = _data 
    spectrograms, labels = spectrograms.to(device), labels.to(device)

    output = model(spectrograms)  # (batch, time, n_class)
    output = F.log_softmax(output, dim=2)
    output = output.transpose(0, 1) # (time, batch, n_class)
    print(output)
    break

tensor([[[-6.8706, -5.8052, -1.5170,  ..., -3.5881, -7.9626, -3.0825],
         [-6.9669, -5.8632, -1.4912,  ..., -3.6018, -8.0781, -3.0979],
         [-7.0027, -5.8804, -1.4784,  ..., -3.6025, -8.1237, -3.0946],
         ...,
         [-6.9849, -5.8595, -1.4754,  ..., -3.5694, -8.1116, -3.0736],
         [-6.9258, -5.8116, -1.4878,  ..., -3.5303, -8.0334, -3.0597],
         [-6.7182, -5.6349, -1.5411,  ..., -3.4202, -7.7381, -3.0196]],

        [[-6.5475, -4.1012, -3.1179,  ..., -4.9172, -9.2291, -1.6324],
         [-6.7298, -4.2143, -3.1724,  ..., -5.0657, -9.5914, -1.6428],
         [-6.7719, -4.2363, -3.1883,  ..., -5.0975, -9.6736, -1.6366],
         ...,
         [-6.7724, -4.2350, -3.1901,  ..., -5.0944, -9.6711, -1.6178],
         [-6.7414, -4.2210, -3.1774,  ..., -5.0712, -9.6174, -1.6093],
         [-6.6330, -4.1908, -3.1346,  ..., -5.0021, -9.4601, -1.6129]],

        [[-7.4286, -2.6372, -3.7349,  ..., -5.1281, -9.0110, -0.4579],
         [-7.4181, -2.6538, -3.7386,  ..., -5

In [54]:
labels.size()

torch.Size([8, 247])

In [55]:
output.size()

torch.Size([647, 8, 29])

In [56]:
sample = labels.detach().cpu()[0]
sample

tensor([10.,  1.,  3.,  6., 13., 10.,  6., 23.,  6.,  1., 21.,  9.,  2., 21.,
         1., 21., 16., 16.,  1., 21.,  9.,  6.,  1., 15., 22., 14.,  3.,  6.,
        19.,  1., 16.,  7.,  1., 21.,  9., 16., 20.,  6.,  1., 24.,  9., 16.,
         1.,  9.,  2., 23.,  6.,  1., 17., 19., 16., 14., 10., 20.,  6.,  5.,
         1., 21., 16.,  1.,  5., 16.,  1.,  2., 20.,  1., 11.,  6., 20., 22.,
        20.,  1., 24., 16., 22., 13.,  5.,  1., 10., 20.,  1., 10., 15.,  4.,
        19.,  6.,  2., 20., 10., 15.,  8.,  1., 10.,  7.,  1., 24.,  6.,  1.,
        16., 15.,  4.,  6.,  1.,  9.,  2., 23.,  6.,  1., 20.,  2., 26.,  1.,
         7., 10., 23.,  6.,  1.,  9., 22., 15.,  5., 19.,  6.,  5.,  1., 20.,
        22.,  4.,  9.,  1.,  5., 10., 20.,  4., 10., 17., 13.,  6., 20.,  1.,
        10., 15.,  1., 19.,  2., 26., 14., 16., 15.,  5.,  1., 21.,  9.,  6.,
         1., 20.,  2., 13., 16., 16., 15.,  1., 10., 20.,  1.,  5., 16., 16.,
        14.,  6.,  5.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  

In [57]:
isample = [int(x.item()) for x in sample]

In [58]:
text_transform.int_to_text(isample)

"i believe that too the number of those who have promised to do as jesus would is increasing if we once have say five hundred such disciples in raymond the saloon is doomed''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''"

In [68]:
text_transform.index_map['28']

KeyError: '28'

In [71]:
out_sample = output.transpose(0, 1)

In [73]:
torch.argmax(out_sample, dim=2)

tensor([[ 2,  9, 28,  ..., 28, 28, 28],
        [ 2,  9, 28,  ..., 28, 28, 28],
        [ 2,  9, 28,  ..., 28, 28, 28],
        ...,
        [ 2,  9, 28,  ..., 28, 28, 28],
        [ 2,  9, 28,  ..., 28, 28, 28],
        [ 2, 28, 28,  ..., 28, 28, 28]], device='cuda:0')

In [49]:
def GreedyDecoder(output, labels, label_lengths, blank_label=28, collapse_repeated=True):
	arg_maxes = torch.argmax(output, dim=2)
	decodes = []
	targets = []
	for i, args in enumerate(arg_maxes):
		decode = []
		targets.append(text_transform.int_to_text(labels[i][:label_lengths[i]].tolist()))
		for j, index in enumerate(args):
			if index != blank_label:
				if collapse_repeated and j != 0 and index == args[j -1]:
					continue
				decode.append(index.item())
		decodes.append(text_transform.int_to_text(decode))
	return decodes, targets

In [61]:
decodes, targets = GreedyDecoder(output=output.transpose(0, 1), labels=labels, label_lengths=label_lengths)

In [62]:
decodes

['ah', 'ah', 'ah', 'ah', 'ah', 'ah', 'ah', 'a']

In [63]:
targets

['i believe that too the number of those who have promised to do as jesus would is increasing if we once have say five hundred such disciples in raymond the saloon is doomed',
 'take from the fire add the yolks of four eggs beaten with the juice of a lemon and three tablespoonfuls of butter bring to the boil add a dozen parboiled oysters pour over the fish and serve baked perch',
 'we want the best in two races and civilizations in exchange for what we have lost some of us have entered upon every known professional career such as medicine law the ministry education and the sciences',
 "and anyway i'm thankful to have julia take a fancy to anybody it leaves us a great deal freer to do as we like i should think that you would see that yourself oh well said nora laughing the whole thing is not worth quarreling about",
 'and there the comrade knocked upon the door and asked for lodging',
 'as he had not been able to bend down for fear of betraying himself he had not cut the bonds of his le